In [1]:
from os import listdir
from numpy import asarray
from numpy import vstack
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from numpy import savez_compressed

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
def load_images(path, size=(256,512)):
    source_imgs_list, target_imgs_list = list(), list()
    for filename in listdir(path):
        img_pixels = load_img(path + filename, target_size=size)
        img_pixels = img_to_array(img_pixels)
        # The source and target images came as a unique image where half of it is
        # the source and the other half the target.
        source_img, target_img = img_pixels[:, :256], img_pixels[:, 256:]
        source_imgs_list.append(source_img)
        target_imgs_list.append(target_img)
    return [asarray(source_imgs_list), asarray(target_imgs_list)]        

In [3]:
from pathlib import Path

path = "./maps/maps/train/"
if not Path("./maps_256.npz").exists():
    [source_images, target_images] = load_images(path)

In [4]:
filename = 'maps_256.npz'
if not Path("./maps_256.npz").exists():
    savez_compressed(filename, source_images, target_images)

# DISCRIMINATOR

In [5]:
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from matplotlib import pyplot

The architecture we're using for the discriminator is the *U-Net*. The paper clearly states how we should build the discriminator:
 
> For  discriminator  networks,  we  use 70×7 Patch-GAN   [21].Let Ck denote a **4×4 Convolution-InstanceNorm-LeakyReLU layer** with **k filters** and **stride 2**.After the last layer, we apply a convolution to produce a 1 dimensional output.  We do not use InstanceNorm for the first C64 layer.  We use **leaky ReLUs with slope 0.2.**  All our three discriminators have the identical architecture as follows:C64-C128-C256-C51

Also it says this *for every network* :

> All  the  networks  were  trained  from  scratch,  using  the **Adam solver [27] and a learning rate of 0.0002**.  We keep the same learning rate for the first 100 epochs and linearly decay the rate to zero over the next 100 epochs.  **Weights were initialized from a Gaussian distribution with mean 0 and standard deviation 0.02.**

In [6]:
# If we concieve a "block" in this context as the set of operations that are executed repeatedly, 
# then a discriminator block only changes in the number of filters it has, it stride length and 
# the input that it receives. Also, in the first layer the batch normalization musn't be performed.

def discriminator_block(init, num_filters, stride_length, input, apply_batch_norm = True):
    
    output = Conv2D(num_filters, (4,4), strides=stride_length, padding='same', kernel_initializer=init)(input)
    if(apply_batch_norm):
        output = BatchNormalization()(output)
    output = LeakyReLU(alpha=0.2)(output)
    
    return output

![U-Net Photo](https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2019/05/Architecture-of-the-U-Net-Generator-Model-1.png)

In [7]:
# As Input to the discriminator a source image and its target image are passed and then concatenated.
# The target image is the one we're trying to generate so it can either be a real or a fake one.
# The source image will remain always the same and condition the discriminator network.

def discriminator(image_shape):
    
    kernel_initializer = RandomNormal(stddev=0.02)
    
    input_source_image = Input(shape=image_shape)
    input_target_image = Input(shape=image_shape)
    
    merged_inputs = Concatenate()([input_source_image, input_target_image])
    
    dis_output = discriminator_block(kernel_initializer, 64,  (2,2), merged_inputs, False)
    dis_output = discriminator_block(kernel_initializer, 128, (2,2), dis_output)
    dis_output = discriminator_block(kernel_initializer, 256, (2,2), dis_output)
    dis_output = discriminator_block(kernel_initializer, 512, (2,2), dis_output)
    dis_output = discriminator_block(kernel_initializer, 512, (1,1), dis_output)
    
    # Here we're using as a final output activation layer a Sigmoid function because we want to generate
    # an image of patches where each patch is a number between 0 or 1 representing the likelihood of that 
    # patch being real or fake (generated).
    dis_output = Conv2D(1, (4,4), padding='same', kernel_initializer=kernel_initializer)(dis_output)
    patch_out = Activation('sigmoid')(dis_output)
    
    model = Model([input_source_image, input_target_image], patch_out)
    
    optimizer_technique = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=optimizer_technique, loss_weights=[0.5])
    
    return model

# ENCODER

![PatchGAN](https://blogs.rstudio.com/tensorflow/posts/2018-09-20-eager-pix2pix/images/unet.png)

In [8]:
def encoder_block(input, num_filters, apply_batch_norm=True):
    
    init = RandomNormal(stddev=0.02)
    
    enc_output = Conv2D(num_filters, (4,4), strides=(2,2), padding="same", kernel_initializer=init)(input)
    if(apply_batch_norm):
        enc_output = BatchNormalization()(enc_output, training=True)
    enc_output = LeakyReLU(alpha=0.2)(enc_output)

    return enc_output

In [9]:
# The decoder blocks take as input among others the skip connections with the encoders. This skip connections
# are then merged with the output of the normalized deconvoluted input.
def decoder_block(input, skip_connection, num_filters, apply_dropout=True):
    
    init = RandomNormal(stddev=0.02)
    
    dec_output = Conv2DTranspose(num_filters, (4,4), strides=(2,2), padding="same", kernel_initializer=init)(input)
    dec_output = BatchNormalization()(dec_output, training=True)
    if(apply_dropout):
        dec_output = Dropout(0.5)(dec_output, training=True)
    dec_output = Concatenate()([dec_output, skip_connection])
    dec_output = Activation('relu')(dec_output)
    

    return dec_output

In [ ]:
# The generator takes as input an image, this image will be the source image or (in our case) a semantic
# labeling map from which we want to obtain a realistic image (target image).
def generator(image_shape=(256,256,3)):
    
    init = RandomNormal(stddev=0.02)
    input_image = Input(shape=image_shape)
    
    encoder_output_1 = encoder_block(input_image, 64, apply_batch_norm=False)
    encoder_output_2 = encoder_block(encoder_output_1 , 128)
    encoder_output_3 = encoder_block(encoder_output_2, 256)
    encoder_output_4 = encoder_block(encoder_output_3, 512)
    encoder_output_5 = encoder_block(encoder_output_4, 512)
    encoder_output_6 = encoder_block(encoder_output_5, 512)
    encoder_output_7 = encoder_block(encoder_output_6, 512)
    
    bottleneck = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(encoder_output_7)
    bottleneck = Activation('relu')(bottleneck)
    
    decoder_output_1 = decoder_block(bottleneck, encoder_output_7, 512)
    decoder_output_2 = decoder_block(decoder_output_1, encoder_output_6, 512)
    decoder_output_3 = decoder_block(decoder_output_2, encoder_output_5, 512)
    decoder_output_4 = decoder_block(decoder_output_3, encoder_output_4, 512)
    decoder_output_5 = decoder_block(decoder_output_4, encoder_output_3, 256, apply_dropout=False)
    decoder_output_6 = decoder_block(decoder_output_5, encoder_output_2, 128, apply_dropout=False)
    decoder_output_7 = decoder_block(decoder_output_6, encoder_output_1, 64,  apply_dropout=False)
    
    generator_output = Conv2DTranspose(3, (4,4), strides=(2,2), padding="same", kernel_initializer=init)(decoder_output_7)
    output_image = Activation('tanh')(generator_output)
    
    model = Model(input_image, output_image)
    return model
    

In [11]:
def gan(generator, discriminator, image_shape):
    
    discriminator.trainable = False
    
    input_source_img = Input(shape=image_shape)
    
    generator_output = generator(input_source_img)
    discriminator_output = discriminator([input_source_img, generator_output])
    
    model = Model(input_source_img, [discriminator_output, generator_output])
    optimizer_technique = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'mae'], optimizer=optimizer_technique, loss_weights=[1,100])
    
    return model
    

In [12]:
def load_real_samples(filename):
    data = load(filename)
    source_imgs , target_imgs = data['arr_0'], data['arr_1']
    source_imgs = (source_imgs - 127.5) / 127.5
    target_imgs = (target_imgs - 127.5) / 127.5
    return [source_imgs, target_imgs]

In [13]:
def retrieve_real_samples(dataset, n_samples, patch_shape):
    source_imgs, target_imgs = dataset
    random_index = randint(0, source_imgs.shape[0], n_samples)
    source_imgs, target_imgs = source_imgs[random_index], target_imgs[random_index]
    real_img_labels = ones((n_samples, patch_shape, patch_shape, 1))
    return [source_imgs, target_imgs], real_img_labels

In [14]:
def generate_fake_samples(generator, samples, patch_shape):
    fake_instances = generator.predict(samples)
    fake_img_labels = zeros((len(fake_instances), patch_shape, patch_shape, 1))
    return fake_instances, fake_img_labels

In [15]:
def scale_img_pixels(img):
    return (img + 1) / 2.0

In [ ]:
# TODO: Separate functionality in this function, you're not just summarizing the performance
# but also saving the model. Split that.
def summarize_performance(step, generator, dataset, n_samples=3):
    
    [source_imgs, target_imgs], _ = retrieve_real_samples(dataset, n_samples, 1)
    fake_imgs, _ = generate_fake_samples(generator, source_imgs, 1)
    
    source_imgs = scale_img_pixels(source_imgs)
    target_imgs = scale_img_pixels(target_imgs)
    fake_imgs = scale_img_pixels(fake_imgs)
    
    for i in range(n_samples):
        pyplot.subplot(3, n_samples, 1 + n_samples + i)
        pyplot.axis('off')
        pyplot.imshow(source_imgs[i])
    
    for i in range(n_samples):
        pyplot.subplot(3, n_samples, 1 + n_samples + i)
        pyplot.axis('off')
        pyplot.imshow(fake_imgs[i])
    
    for i in range(n_samples):
        pyplot.subplot(3, n_samples, 1 + n_samples*2 + i)
        pyplot.axis('off')
        pyplot.imshow(target_imgs[i])
    
    filename1 = 'plot_%06d.png' % (step + 1)
    pyplot.savefig(filename1)
    pyplot.close()
    
    # Yo quitaría esto de aquí
    filename2 = 'model_%06d.h5' % (step + 1)
    generator.save(filename2)
    print('Saved: %s and %s' % (filename1, filename2))
    

In [17]:
def train(discriminator, generator, gan, dataset, n_epochs=100, n_batch=1):
    
    discriminator_patch_shape = discriminator.output_shape[1]
    source_imgs, target_imgs = dataset
    iterations = int(len(source_imgs) / n_batch)
    n_steps = iterations * n_epochs
    
    for i in range(n_steps):
        [source_imgs, target_imgs], real_labels = retrieve_real_samples(dataset, n_batch, discriminator_patch_shape)
        fake_imgs, fake_labels = generate_fake_samples(generator, source_imgs, discriminator_patch_shape)
        disc_loss_real = discriminator.train_on_batch([source_imgs, target_imgs], real_labels)
        disc_loss_fake = discriminator.train_on_batch([source_imgs, fake_imgs], fake_labels)
        gan_loss, _, _ = gan.train_on_batch(source_imgs, [real_labels, target_imgs])
        
        print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, disc_loss_real, disc_loss_fake, gan_loss))
        if (i % 10) == 0:
            summarize_performance(i, generator, dataset)

In [18]:
dataset = load_real_samples('maps_256.npz')

In [19]:
image_shape = dataset[0].shape[1:]
image_shape

(256, 256, 3)

In [20]:
discriminator = discriminator(image_shape)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [21]:
generator = generator(image_shape)

In [22]:
gan = gan(generator, discriminator, image_shape)

In [23]:
summarize_performance(1, generator, dataset)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  app.launch_new_instance()


Saved: plot_000002.png and model_000002.h5


In [24]:
train(discriminator, generator, gan, dataset, 1, 1)

C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>1, d1[0.244] d2[0.982] g[78.605]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  app.launch_new_instance()


Saved: plot_000001.png and model_000001.h5
>2, d1[0.208] d2[0.702] g[67.284]
>3, d1[0.206] d2[0.606] g[61.548]
>4, d1[0.357] d2[0.421] g[74.253]
>5, d1[0.364] d2[0.439] g[72.081]
>6, d1[0.313] d2[0.409] g[68.554]
>7, d1[0.219] d2[0.358] g[65.199]
>8, d1[0.086] d2[0.233] g[49.261]
>9, d1[0.320] d2[0.232] g[61.505]
>10, d1[0.394] d2[0.256] g[52.505]
>11, d1[0.107] d2[0.247] g[56.560]
Saved: plot_000011.png and model_000011.h5
>12, d1[0.045] d2[0.284] g[53.953]
>13, d1[0.091] d2[0.132] g[52.444]
>14, d1[0.245] d2[0.328] g[29.835]
>15, d1[0.118] d2[0.105] g[40.900]
>16, d1[0.172] d2[0.149] g[31.991]
>17, d1[0.171] d2[0.066] g[27.440]
>18, d1[0.274] d2[0.153] g[33.529]
>19, d1[0.028] d2[0.069] g[41.253]
>20, d1[0.439] d2[0.245] g[26.894]
>21, d1[0.051] d2[0.024] g[33.966]
Saved: plot_000021.png and model_000021.h5
>22, d1[0.017] d2[0.016] g[36.236]
>23, d1[0.015] d2[0.052] g[35.805]
>24, d1[0.062] d2[0.062] g[31.890]
>25, d1[0.009] d2[0.060] g[29.056]
>26, d1[0.049] d2[0.052] g[29.113]
>27,

KeyboardInterrupt: 